# **Module 2 Homework**

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

---

## **Install**


In [28]:
# !pip install yfinance
# !pip install pandas_datareader
# !pip install pyarrow
# !pip install fastparquet

## **Imports**

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import plotly.graph_objs as go
import plotly.express as px
import time
from datetime import date
import matplotlib.pyplot as plt
import requests

---
## **Question 1: IPO Filings Web Scraping and Data Processing**  

**What's the total sum ($m) of 2023 filings that happened on Fridays?**  

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/  
Convert the **'Filing Date'** to datetime(), **'Shares Offered'** to float64 (if **'-'** is encountered, populate with NaNs).  
Define a new field **'Avg_price'** based on the **"Price Range"**, which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).  
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to **"Shares Offered"** * **"Avg_price"*** (when both columns are defined; otherwise, it's NaN)  

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`).   
You should see 32 records in total, 25 of it is not null.  

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
     
ipo_df = ipo_dfs[0]
     
# Convert the 'Filing Date' to datetime()
ipo_df['Filing Date'] = pd.to_datetime(ipo_df['Filing Date'], format='mixed')
ipo_df.head()

/tmp/ipykernel_2181/2058199188.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-


In [3]:
# Convert 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
ipo_df['Shares Offered'] = ipo_df['Shares Offered'].replace('-', np.NaN).astype('float64')
     

ipo_df.dtypes

Filing Date       datetime64[ns]
Symbol                    object
Company Name              object
Price Range               object
Shares Offered           float64
dtype: object

In [4]:
# Define a new field 'Avg_price' based on the "Price Range",
# which equals to NaN if no price is specified,
# to the price (if only one number is provided),
# or to the average of 2 prices (if a range is given)

def calc_avg_price(price_range):
    if price_range == '-':
        return np.NaN
    elif price_range.find(' - ') == -1:
        return float(price_range[price_range.find('$')+1:])
    else:
        first_num = float(price_range[price_range.find('$')+1:price_range.find(' ')])
        second_num = float(price_range[price_range.rfind('$') + 1:])
        return (first_num + second_num) / 2
    

ipo_df['Avg_price'] = ipo_df['Price Range'].apply(lambda x: calc_avg_price(x))
ipo_df['Avg_price']

0        NaN
1       3.00
2      10.00
3      10.00
4        NaN
       ...  
321     9.00
322    10.00
323     9.00
324     4.00
325     5.75
Name: Avg_price, Length: 326, dtype: float64

In [5]:
# Define a column "Shares_offered_value",
# which equals to "Shares Offered" * "Avg_price"
# (when both columns are defined; otherwise, it's NaN)

ipo_df['Shares_offered_value'] = ipo_df['Shares Offered'] * ipo_df['Avg_price']
ipo_df['Shares_offered_value']

0             NaN
1      11250000.0
2      60000000.0
3      50000000.0
4             NaN
          ...    
321    13500000.0
322    50000000.0
323    22500000.0
324    17200000.0
325    11500000.0
Name: Shares_offered_value, Length: 326, dtype: float64

In [6]:

# Find the total sum in $m (millions of USD, closest INTEGER number)
# for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4).
# You should see 32 records in total, 24 of it is not null.
     

ipo_df_2023 = ipo_df[ipo_df['Filing Date'].dt.year == 2023]
     

ipo_df_2023_fri = ipo_df[(ipo_df['Filing Date'].dt.year == 2023) & (ipo_df['Filing Date'].dt.day_of_week == 4)]
     

ipo_df_2023_fri.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 166
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Avg_price             25 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 2.0+ KB


In [9]:
ipo_df_2023_fri.Shares_offered_value.sum() / 1000000

285.7

---
### **Question 2:  IPOs "Fixed days hold" strategy**


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).   
Get all OHLCV daily prices for all stocks with an **"IPO date"** before March 1, 2024 ("< 2024-03-01") - 184 tickers (without **'RYZB'**).   
Please remove **'RYZB'**, as it is no longer available on Yahoo Finance.  

Sometimes you may need to adjust the symbol name (e.g., **'IBAC'** on stockanalysis.com -> **'IBACU'** on Yahoo Finance) to locate OHLCV prices for all stocks.  
Also, you can see the ticker changes using this [link](https://stockanalysis.com/actions/changes/).  
Some of the tickers (like **'DYCQ'** and **'LEGT'**) were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.  

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the **"Adj. Close"** price in X days (e.g., if X=1, you sell on the next day).  
Find X, when the 75% quantile growth (among 185 investments) is the highest.   

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo   Finance), and perform vector operations on the resulting dataset.  
* You can use the **`DataFrame.describe()`** function to get mean, min, max, 25-50-75% quantiles.  


Additional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.  
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?   

In [ ]:
url_2023 = "https://stockanalysis.com/ipos/2023/"
response_2023 = requests.get(url_2023, headers=headers)
ipo_2023_dfs = pd.read_html(response_2023.text)
ipos_2023 = ipo_2023_dfs[0]

url_2024 = "https://stockanalysis.com/ipos/2024/"
response_2024 = requests.get(url_2024, headers=headers)
ipo_2024_dfs = pd.read_html(response_2024.text)
ipos_2024 = ipo_2024_dfs[0]

stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'], format='%b %d, %Y')
stacked_ipos_df = stacked_ipos_df[stacked_ipos_df['IPO Date'] < "2024-03-01"]
stacked_ipos_df = stacked_ipos_df[stacked_ipos_df['Symbol'] != "RYZB"]

In [11]:
ALL_TICKERS = list(stacked_ipos_df["Symbol"])
#after some sort of merger/acquisiton, PTHR has become HOVR
PTHR_index = ALL_TICKERS.index("PTHR")
ALL_TICKERS[PTHR_index] = "HOVR"
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # future returns
  for i in reversed(range(-30,0)):
    historyPrices['growth_future_'+str(i*-1)+'d'] = historyPrices['Adj Close'].shift(i) / historyPrices['Adj Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  #historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  #historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  #historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  #historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  #historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  #historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 SMXT


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 VHAI



[*********************100%%**********************]  1 of 1 completed

2 DYCQ


...

181 SKWD



[*********************100%%**********************]  1 of 1 completed

182 ISRL



[*********************100%%**********************]  1 of 1 completed

183 MGOL


In [12]:
cols = [i for i in stocks_df.keys() if i.find('growth')>=0]
stocks_df_growth = stocks_df[cols]
stocks_df_growth_desc = stocks_df_growth.describe()
stocks_df_growth_desc.loc['75%'].idxmax()

'growth_future_30d'

---
### **Question 3: Is Growth Concentrated in the Largest Stocks?**

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**   


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks   
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):  

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`  

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`  

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`  

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`  
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`  

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`  

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`  

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`  

You should be able to obtain stats for 33 **LARGEST STOCKS** and 32 **LARGE STOCKS** (from the actual stats on Yahoo Finance)  

Calculate  **`growth_7d`** for every stock and every day.  
Get the average daily **`growth_7d`** for the **LARGEST_STOCKS group** vs. the **LARGE_STOCKS group**.  

For example, for the first of data you should have:  
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).  

Calculate the number of days when the **LARGE GROUP** (new smaller stocks) outperforms the **LARGEST GROUP**, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)


In [7]:
# GEt the LARGEST STOCK companies
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM'] #11

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA'] #11

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS'] #11

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS
print('Num stocks in LARGEST STOCKS: ',len(LARGEST_STOCKS))

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL'] #11

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA'] #12

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS'] #11

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA
print('Num stocks in LARGE STOCKS: ',len(LARGE_STOCKS))

Num stocks in LARGEST STOCKS:  33
Num stocks in LARGE STOCKS:  34


In [8]:
large_stocks_str = " ".join(LARGE_STOCKS)
large_ohlcv = yf.download(large_stocks_str, period="max", #start="2014-01-01", end="2023-12-31",
                          interval="1d")
large_ohlcv.info()

largest_stocks_str = " ".join(LARGEST_STOCKS)
largest_ohlcv = yf.download(largest_stocks_str, period="max", #start="2014-01-01", end="2023-12-31",
                         interval="1d") #period="max",
largest_ohlcv.info()

[*********************100%%**********************]  33 of 33 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15966 entries, 1962-01-02 to 2024-05-09
Columns: 198 entries, ('Adj Close', 'ADANIENT.NS') to ('Volume', 'XOM')
dtypes: float64(198)
memory usage: 24.2 MB


[*********************100%%**********************]  33 of 33 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13358 entries, 1972-06-01 to 2024-05-09
Columns: 198 entries, ('Adj Close', 'AAPL') to ('Volume', 'V')
dtypes: float64(198)
memory usage: 20.3 MB


In [9]:
# Keep only the Adj Close
large_ohlcv =large_ohlcv.loc[:, ['Adj Close']]
largest_ohlcv =largest_ohlcv.loc[:, ['Adj Close']]
large_ohlcv,largest_ohlcv

(Price         Adj Close                                                    \
 Ticker      ADANIENT.NS       AI.PA      AIR.PA      ALV.DE BAJFINANCE.NS   
 Date                                                                        
 1962-01-02          NaN         NaN         NaN         NaN           NaN   
 1962-01-03          NaN         NaN         NaN         NaN           NaN   
 1962-01-04          NaN         NaN         NaN         NaN           NaN   
 1962-01-05          NaN         NaN         NaN         NaN           NaN   
 1962-01-08          NaN         NaN         NaN         NaN           NaN   
 ...                 ...         ...         ...         ...           ...   
 2024-05-03  2993.250000  180.619995  154.339996  250.820343   6931.500000   
 2024-05-06  2874.649902  181.899994  156.660004  255.567139   6859.850098   
 2024-05-07  2835.949951  183.979996  158.860001  258.700012   6872.549805   
 2024-05-08  2860.550049  186.539993  161.520004  259.744293   6

In [10]:
large_ohlcv = pd.concat([large_ohlcv,
                         pd.concat({f'growth_7d': large_ohlcv['Adj Close'] / large_ohlcv['Adj Close'].shift(7)}, axis=1)
                         ], axis=1)
largest_ohlcv = pd.concat([largest_ohlcv,
                         pd.concat({f'growth_7d': largest_ohlcv['Adj Close'] / largest_ohlcv['Adj Close'].shift(7)}, axis=1)
                         ], axis=1)

large_ohlcv.head()
largest_ohlcv.head()

Adj Close                                                         \
Ticker          AAPL ACN AMZN ASML AVGO BHARTIARTL.NS BRK-B CDI.PA GOOG HDB   
Date                                                                          
1972-06-01       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-02       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-05       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-06       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   
1972-06-07       NaN NaN  NaN  NaN  NaN           NaN   NaN    NaN  NaN NaN   

            ... growth_7d                                                     \
Ticker      ...       NVO OR.PA RELIANCE.NS RMS.PA SAP SBIN.NS SIE.DE TCS.NS   
Date        ...                                                                
1972-06-01  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-02  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-05  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-06  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   
1972-06-07  ...       NaN   NaN         NaN    NaN NaN     NaN    NaN    NaN   

                    
Ticker     TTE   V  
Date                
1972-06-01 NaN NaN  
1972-06-02 NaN NaN  
1972-06-05 NaN NaN  
1972-06-06 NaN NaN  
1972-06-07 NaN NaN  

[5 rows x 66 columns]

In [11]:
count_days=len(large_ohlcv)
print(count_days)

15966


In [12]:
large_ohlcv.head()

Adj Close                                                    \
Ticker     ADANIENT.NS AI.PA AIR.PA ALV.DE BAJFINANCE.NS BUD CDI.PA COST   
Date                                                                       
1962-01-02         NaN   NaN    NaN    NaN           NaN NaN    NaN  NaN   
1962-01-03         NaN   NaN    NaN    NaN           NaN NaN    NaN  NaN   
1962-01-04         NaN   NaN    NaN    NaN           NaN NaN    NaN  NaN   
1962-01-05         NaN   NaN    NaN    NaN           NaN NaN    NaN  NaN   
1962-01-08         NaN   NaN    NaN    NaN           NaN NaN    NaN  NaN   

                         ... growth_7d                                       \
Ticker     DTE.DE EL.PA  ...    PRX.AS SNY SU.PA SUNPHARMA.NS TATAMOTORS.NS   
Date                     ...                                                  
1962-01-02    NaN   NaN  ...       NaN NaN   NaN          NaN           NaN   
1962-01-03    NaN   NaN  ...       NaN NaN   NaN          NaN           NaN   
1962-01-04    NaN   NaN  ...       NaN NaN   NaN          NaN           NaN   
1962-01-05    NaN   NaN  ...       NaN NaN   NaN          NaN           NaN   
1962-01-08    NaN   NaN  ...       NaN NaN   NaN          NaN           NaN   

                                      
Ticker     TITAN.NS TSLA UNH WMT XOM  
Date                                  
1962-01-02      NaN  NaN NaN NaN NaN  
1962-01-03      NaN  NaN NaN NaN NaN  
1962-01-04      NaN  NaN NaN NaN NaN  
1962-01-05      NaN  NaN NaN NaN NaN  
1962-01-08      NaN  NaN NaN NaN NaN  

[5 rows x 66 columns]

In [13]:
(
        large_ohlcv
        .loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_large"})
).head()

/tmp/ipykernel_2181/2591226654.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  large_ohlcv


,growth_7d_large
Date,
2014-01-01,1.009288
2014-01-02,0.999349
2014-01-03,0.999633
2014-01-06,0.990707
2014-01-07,0.990938


In [14]:
# Create a dataframe containing growth_7d for LARGE and LARGEST stocks
(
  pd.concat([
    (
        large_ohlcv
        .loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_large"})
    ),
    (
        largest_ohlcv
        .loc[(largest_ohlcv.index >= "2014-01-01") & (largest_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_largest"})
    )], axis=1
  )
  .loc['2014-01-01']
)

/tmp/ipykernel_2181/727573092.py:5: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  large_ohlcv
/tmp/ipykernel_2181/727573092.py:11: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  largest_ohlcv


growth_7d_large      1.009288
growth_7d_largest    1.011797
Name: 2014-01-01 00:00:00, dtype: float64

In [15]:
# Compare and extract how many times growth_7d for LARGE STOCKS is higher than for LARGEST STOCKS
times_large_gt_largest = (
  pd.concat([
    (
        large_ohlcv
        .loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_large"})
    ),
    (
        largest_ohlcv
        .loc[(largest_ohlcv.index >= "2014-01-01") & (largest_ohlcv.index <= "2023-12-31"),['growth_7d']]
        .groupby(level=0, axis=1).mean()
        .rename(columns={"growth_7d": "growth_7d_largest"})
    )], axis=1
  )
  .loc[lambda df: df['growth_7d_large'] > df['growth_7d_largest']]
  .count()
)[0]

print("Total days: ",len(large_ohlcv.loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31")].index))
print("% days LARGE STOCKS is higher than LARGEST STOCKS: ",np.rint((times_large_gt_largest / len(large_ohlcv.loc[(large_ohlcv.index >= "2014-01-01") & (large_ohlcv.index <= "2023-12-31")].index))*100))

Total days:  2595
% days LARGE STOCKS is higher than LARGEST STOCKS:  47.0


/tmp/ipykernel_2181/1184923950.py:5: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  large_ohlcv
/tmp/ipykernel_2181/1184923950.py:11: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  largest_ohlcv
/tmp/ipykernel_2181/1184923950.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  times_large_gt_largest = (


---
### **Question 4: Trying Another Technical Indicators strategy**

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**


First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31).  
If you encounter any difficulties running the Colab - you can download it using this [link](https://drive.google.com/file/d/1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb/view?usp=sharing).

Let's assume you've learned about the awesome **CCI indicator** ([Commodity Channel Index](https://www.investopedia.com/terms/c/commoditychannelindex.asp)), and decided to use only it for your operations.  

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (`Date.dt.dayofweek()==4`).  

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.  

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years?  
One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit.  
You need to sum these results over all trades (460 times in 10 years).  

Additional:  
  * Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")  
  * are you still profitable on those trades?

In [16]:
import pyarrow
import fastparquet

file = "/workspaces/Stock_Markets_Analytics_ZoomCamp/data/stocks_df_combined_trunc_2014_2023.parquet.brotli"
df = pd.read_parquet(file, engine='pyarrow')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80762 entries, 7011 to 5342
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 105.4+ MB


In [17]:
print(list(df.columns))

['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker', 'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d', 'growth_7d', 'growth_30d', 'growth_90d', 'growth_365d', 'growth_future_5d', 'SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative', 'volatility', 'is_positive_growth_5d_future', 'ticker_type', 'index_x', 'adx', 'adxr', 'apo', 'aroon_1', 'aroon_2', 'aroonosc', 'bop', 'cci', 'cmo', 'dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext', 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix', 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo', 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk', 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr', 'index_y', 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase', 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine', 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice', 'index', 'cdl2crows', 'cdl3blackrows', 'c

In [35]:
df.head()

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Define the start and end dates
start_date = '2014-01-01'
end_date = '2023-12-31'

# Filter the DataFrame
df_filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

trades = df_filtered[(df_filtered['cci'] > 200) & (df_filtered['Date'].dt.dayofweek == 4)].copy()

In [19]:
len(trades)

460

In [20]:
# Calculate the profit for each trade using growth_future_5d
trades.loc[:, 'profit'] = 1000 * (trades['growth_future_5d'] - 1)

# Sum up the profits and convert to thousands, then round
total_profit_thousands = round(trades['profit'].sum() / 1000)

In [21]:
total_profit_thousands

1

With calculator, the costs are about $2,000

![fees](https://github.com/oussou-dev/Stock_Markets_Analytics_ZoomCamp/blob/main/img/fees_calculation.png?raw=true)

---
### **[EXPLORATORY] Question 5: Finding Your Strategy for IPOs**

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.  

How would you correct/refine the approach?   

Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?  

E.g. (some ideas) Do you want to focus on the specific vertical?   

Do you want to build a smart comparison vs. existing stocks on the market?   

Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?

To refine the approach for IPO investments, we can take several steps:

1. **Sector Analysis**: Investigate specific sectors or industries that historically perform better in IPOs. This can involve analyzing trends, market demand, and growth potential within each sector. Publicly available data from financial news sources, industry reports, and market research firms can provide insights into sector performance.

2. **Financial Metrics**: Focus on key financial metrics beyond just median and average investments. Look at metrics such as revenue growth, profit margins, earnings per share (EPS), and valuation multiples. Comparing these metrics against industry benchmarks and peer companies can help identify IPOs with strong financial fundamentals.

3. **Management Team Evaluation**: Assess the quality and experience of the management team behind the IPO. Look for leadership with a track record of success in similar ventures or relevant industry experience. Publicly available information such as executive bios, past company performance, and industry reputation can aid in this evaluation.

4. **Market Conditions**: Consider broader market conditions and sentiment. Evaluate factors such as overall economic health, market volatility, and investor sentiment towards IPOs. This can help gauge the timing of investments and whether market conditions are favorable for new offerings.

5. **Regulatory and Legal Considerations**: Stay informed about regulatory changes and legal implications that may impact IPOs. This includes compliance requirements, regulatory scrutiny, and potential legal risks associated with the company or industry.

6. **Comparative Analysis**: Conduct a comparative analysis of the IPO against existing publicly traded companies within the same sector. Compare financial performance, market positioning, competitive advantages, and growth prospects to identify potential outliers and assess relative value.

7. **Qualitative Factors**: Incorporate qualitative factors such as product differentiation, market innovation, and competitive advantage. Analyze the company's business model, technology, intellectual property, and market positioning to assess long-term growth potential and sustainability.

By incorporating these steps and leveraging publicly available data, we can refine the approach to IPO investments and make more informed decisions.